In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text
import pandas as pd
import re

# QUERIES

In [2]:
def query_get_all_values(data, text):
    """
    Return all the values for that material
    """
    numeros = re.findall(r'\d+', text)
    trobat = False
    for n in numeros:
        if int(n) in data["material"].unique():
            trobat = int(n)
            break
    if trobat == False:
        return "Si us plau, escriu l'index del material que vols al text"
    else:
        entrada = trobat

    proportion_devaluation = data[data["material"] == int(entrada)]
    return proportion_devaluation

def query_get_autoarima_index(data, text):
    numeros = re.findall(r'\d+', text)
    trobat = False
    for n in numeros:
        if int(n) in data["material"].unique():
            trobat = int(n)
            break
    """
    Return forecast index for that material
    """
    if trobat == False:
        return "Si us plau, escriu l'index del material que vols al text"
    else:
        entrada = trobat

    if int(entrada) in data["material"].unique():
        proportion_devaluation = data[data["material"] == int(entrada)]["auto_arima_index"].values[0]
        return f"El càlcul usant la predicció de sèries temporals és de {proportion_devaluation}"
    else:
        return "Ho sento, ho he trobat el material. Estàs segur que és l'índex correcte?"

def query_get_autoencoder_index(data, text):
    numeros = re.findall(r'\d+', text)
    trobat = False
    for n in numeros:
        if int(n) in data["material"].unique():
            trobat = int(n)
            break
    """
    Return forecast index for that material
    """
    if trobat == False:
         return "Si us plau, escriu l'index del material que vols al text"
    else:
        entrada = trobat

    if int(entrada) in data["material"].unique():
        proportion_devaluation = data[data["material"] == int(entrada)]["auto_encoder_index"].values[0]
        return f"El càlcul usant l'auto encoder (xarxa neuronal) és de {proportion_devaluation}"
    else:
        return "Ho sento, ho he trobat el material. Estàs segur que és l'índex correcte?"

def query_get_prediction(data, text):
    numeros = re.findall(r'\d+', text)
    trobat = False
    for n in numeros:
        if int(n) in data["material"].unique():
            trobat = int(n)
            break
    """
    Return forecast index for that material
    """
    if trobat == False:
        return "Si us plau, escriu l'index del material que vols al text"
    else:
        entrada = trobat

    if int(entrada) in data["material"].unique():
        proportion_devaluation = data[data["material"] == int(entrada)]["fair_price"].values[0]
        return f"El valor just estimat pel model és {proportion_devaluation}"
    else:
        return "Ho sento, ho he trobat el material. Estàs segur que és l'índex correcte?"

def query_get_devaluation(data, text):
    numeros = re.findall(r'\d+', text)
    trobat = False
    for n in numeros:
        if int(n) in data["material"].unique():
            trobat = int(n)
            break
    """
    Return forecast index for that material
    """
    if trobat == False:
        return "Si us plau, escriu l'index del material que vols al text"
    else:
        entrada = trobat

    if int(entrada) in data["material"].unique():
        element = data[data["material"] == int(entrada)].iloc[0]
        if element["fair_price"] == element["new_value"]:
            return f"S'ha devaluat un {element['fair_price'] / element['cost_unitari_stock_2023']} respecte al cost"
        else:
            return f"No s'ha devaluat. El valor predit era {element['fair_price']}, mentre que el cost  {element['cost_unitari_stock_2023']}"
    else:
        return "Ho sento, ho he trobat el material. Estàs segur que és l'índex correcte?"
    
def query_get_total_devaluation(data, text):
    total_value = 0
    count = 0
    for index, element in data[data["fair_price"] == data["new_value"]].iterrows():
        total_value += element["valor_total_stock_2023"] - element["stock_final_2023"] * element["fair_price"]
        count += 1
    return(f"Valor total devaluat: {total_value}, en un total de {count} elements")

def query_text(data, text):
    return "Digues quina informació de la base de dades vols. Per exemple, dona'm el percentatge de devaluació del material 100022432"


In [3]:
queries = {
    "Dona'm el percentatge de devaluació del material 100022432": query_get_devaluation,
    "Retorna tots els valors del material 100022432": query_get_all_values,
    "Retorna l'índex de sèries temporals de deteriorament per al material 100022432 amb ARIMA": query_get_autoarima_index,
    "Retorna l'índex amb autoencoder de deteriorament per al material 100022432 amb autoencoder": query_get_autoencoder_index,
    "Retorna la predicció per al material 100022432, valor just, estimat": query_get_prediction,
    "Retorna el valor total de devaluació de la base de dades": query_get_total_devaluation,
    "Hola, què saps fer, que tal, provant": query_text
}

In [4]:
import pickle
with open("queries.pkl", "wb") as f:
    pickle.dump(queries, f)

# MODEL DE LLENGUATGE - carrega

In [5]:
model_path = './universal_sentence_encoder.tflite'

data = pd.read_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/results_inventory_impairment.xlsx")

import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
TextEmbedder = mp.tasks.text.TextEmbedder
TextEmbedderOptions = mp.tasks.text.TextEmbedderOptions

# For creating a text embedder instance:
options = TextEmbedderOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    quantize=True)
text_embedder = TextEmbedder.create_from_options(options)

def embed_text(text):
    return text_embedder.embed(text).embeddings[0]


In [6]:
with open("queries.pkl", "rb") as f:
    loaded_queries = pickle.load(f)

query_embeddings = [text_embedder.embed(query).embeddings[0] for query in queries.keys()]

In [7]:
max_sim = float("-inf")
for i, query in enumerate(query_embeddings):
    similarities =  TextEmbedder.cosine_similarity(query_embeddings[0], query)
    if similarities > max_sim:
        q = i
        max_sim = similarities

In [8]:
queries[list(queries.keys())[q]](data, "hola")

"Si us plau, escriu l'index del material que vols al text"

# CHAT

In [9]:
from ipywidgets import widgets
from IPython.display import display, HTML, clear_output

class Chat:
    def __init__(self):
        self.input_box = widgets.Text(placeholder="Type your message and press Enter")
        self.clear_button = widgets.Button(description="Clear Chat")
        self.output_area = widgets.Output()
        self.setup_ui()

    def setup_ui(self):
        self.input_box.on_submit(self.send_message)
        self.clear_button.on_click(self.clear_chat)
        display(widgets.VBox([self.input_box, self.clear_button]))
        display(self.output_area)
        with self.output_area:
            self.display_chatbot_message("Welcome to the chat interface! Type your message below.")

    def send_message(self, sender):
        message = self.input_box.value.strip()
        if message:
            self.display_user_message(message)
            self.input_box.value = ""

    def display_user_message(self, message):
        message_widget = HTML(f"<strong>You:</strong> {message}<br>")
        with self.output_area:
            display(message_widget)
        self.display_chatbot_message(self.handle_user_message(message))

    def handle_user_message(self, text):
        embed = text_embedder.embed(text).embeddings[0]
        max_sim = float("-inf")
        for i, query in enumerate(query_embeddings):
            similarities =  TextEmbedder.cosine_similarity(embed, query)
            if similarities > max_sim:
                q = i
                max_sim = similarities
        return queries[list(queries.keys())[q]](data, text)
    def display_chatbot_message(self, message):
        message_widget = HTML(f"<strong>ChatBot:</strong> {message}<br>")
        with self.output_area:
            display(message_widget)

    def clear_chat(self, sender):
        self.output_area.clear_output()

    @classmethod
    def startChat(cls):
        chat_instance = cls()
        return chat_instance


# Prova'l

Abans d'executar-ho, asssegura't d'haver executat totes les cel·les anteriors. Tingues en compte que és simplement una versió petita de prova per tal de visualitzar què podria ser capaç de fer un sistema d'aquest estil, i que tampoc acaba d'estar optimitzat per entendre el català del tot bé (tot i que és el llenguatge en el que espera que li parlis). Les respostes que té són predeterminades, però podrien ser també generades per un LLM. Si peta i veus que no torna a sortir el xat, reseteja el notebook.

In [10]:
# Example usage
Chat.startChat()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_28808\2138663488.py:12: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.input_box.on_submit(self.send_message)


Output()

De quin material vols obtenir el percentatge de devaluació?